# Title
[]()

In [1]:

import sys
sys.path.append(r"/home/silvhua/custom_python")
sys.path.append(r'/home/silvhua/repositories/GHL-chat/src/')
from silvhua import *

from openai import OpenAI
import os
from time import time

from langchain.chat_models import ChatOpenAI

# Create memory 
from langchain.memory.chat_message_histories import DynamoDBChatMessageHistory
from langchain.memory import ConversationBufferMemory

from langchain.agents.openai_functions_agent.base import OpenAIFunctionsAgent
from langchain.schema.messages import SystemMessage

from langchain.prompts import MessagesPlaceholder

from langchain.agents import AgentExecutor

from langchain.agents import Tool


# Initialize

In [3]:

conversation_dict = dict()
system_message_dict = dict()
reply_dict = dict()

In [4]:
contacts = load_json('contacts.json', '../src/app/private')

# Iteration 1

In [12]:
business_name = 'Coach McLoone'
''.join([f'{word}_' for word in business_name.split(' ')])[:-1]

'Coach_McLoone'

In [17]:
def create_system_message(
        business_name, prompts_filepath='../prompts',
        examples_filepath='../data/chat_examples', doc_filepath='../data/rag_docs'
        ):
    
    instructions = load_txt(f'{business_name}.md', prompts_filepath)
    examples = load_txt(f'{business_name}.txt', examples_filepath)
    document = load_txt(f'{business_name}_doc.md', doc_filepath)

    system_message = f"""# Stage 1

    {instructions}

    ## Examples

    {examples}

    ## Relevant documentation

    {document}

    # Stage 2

    Review your response from stage 1. 
    Revise your response if needed to make sure you followed the instructions.
    Make sure that if the question cannot be answered through the documentation, 
    you return "[ALERT HUMAN]".
    
    # Stage 3

    Review your response from stage 2 to ensure your response is concise.
    """

    prompt = """
    Write the next OutboundMessage based on the following InboundMessage, 
    which is delimited by triple backticks: ```{InboundMessage}```
    """
    system_message = f'{system_message}{prompt}'
    return system_message

def create_chatbot(contactId, system_message, tools, model="gpt-3.5-turbo-16k", verbose=True):

    llm = ChatOpenAI(
        temperature = 0,
        openai_organization=os.environ['openai_organization'],
        openai_api_key=os.environ['openai_api_key'],
        model=model, 
        model_kwargs={"response_format": {"type": "json_object"}} # https://platform.openai.com/docs/guides/text-generation/json-mode  # https://api.python.langchain.com/en/latest/chat_models/langchain_community.chat_models.openai.ChatOpenAI.html?highlight=chatopenai#
        )
    message_history = DynamoDBChatMessageHistory(
        table_name="SessionTable", session_id=contactId,
        key={
            "SessionId": contactId,
            "type": 'ChatHistory',
            }
        )
    system_message = SystemMessage(
        content=(system_message),
        input_variables=['InboundMessage']
    )
    
    prompt = OpenAIFunctionsAgent.create_prompt(
        system_message=system_message,
        extra_prompt_messages=[
            MessagesPlaceholder(variable_name='chat_history')
            ]
    )

    agent = OpenAIFunctionsAgent(llm=llm, tools=tools, prompt=prompt)
    agent_executor = AgentExecutor(
        agent=agent, tools=tools, 
        verbose=verbose, return_intermediate_steps=True
        )
    agent_info = {
        'agent': agent,
        'agent_executor': agent_executor,
        'chat_history': message_history.messages
    }
    return agent_info

def chat_with_chatbot(user_input, agent_info):
    start_time = time()
    print(f'Chat history length: {len(agent_info["chat_history"])}')
    chat_history = agent_info['chat_history']
    result = agent_info['agent_executor']({
        "input": user_input,
        "chat_history": chat_history
        })
    print(f'Response time: {time() - start_time} seconds')
    
    return result

conversation_id = 1
system_message_dict[conversation_id] = create_system_message(
    'CoachMcloone', prompts_filepath='../prompts',
    examples_filepath='../data/chat_examples', doc_filepath='../data/rag_docs'
)
# conversation_dict[conversation_id] = create_chatbot(
#     contactId, system_message_dict[conversation_id], tools=tools,
#     # model='gpt-4-32k'
#     )

# reply_dict[conversation_id] = chat_with_chatbot(
#     InboundMessage, conversation_dict[conversation_id]
# )
# print(f'Reply from `chat_with_chatbot`: {reply}')
# reply_text = reply_dict[conversation_id]['output']

## 1.1

In [5]:
def create_system_message(
        business_name, prompts_filepath='../prompts',
        examples_filepath='../data/chat_examples', doc_filepath='../data/rag_docs'
        ):
    
    instructions = load_txt(f'{business_name}.md', prompts_filepath)
    examples = load_txt(f'{business_name}.txt', examples_filepath)
    document = load_txt(f'{business_name}_doc.md', doc_filepath)

    system_message = f"""# Stage 1

{instructions}

## Examples

{examples}

## Relevant documentation

{document}

# Stage 2

Review your response from stage 1. 
Revise your response if needed to make sure you followed the instructions.
Make sure that if the question cannot be answered through the documentation, 
you return "[ALERT HUMAN]".

# Stage 3

Review your response from stage 2 to ensure your response is concise.
    """

    prompt = """
    Write the next OutboundMessage based on the following InboundMessage, 
    which is delimited by triple backticks: ```{InboundMessage}```
    """
    system_message = f'{system_message}{prompt}'
    return system_message

def create_chatbot(contactId, system_message, tools, model="gpt-3.5-turbo-16k", verbose=True):

    llm = ChatOpenAI(
        temperature = 0,
        openai_organization=os.environ['openai_organization'],
        openai_api_key=os.environ['openai_api_key'],
        model=model, 
        model_kwargs={"response_format": {"type": "json_object"}} # https://platform.openai.com/docs/guides/text-generation/json-mode  # https://api.python.langchain.com/en/latest/chat_models/langchain_community.chat_models.openai.ChatOpenAI.html?highlight=chatopenai#
        )
    message_history = DynamoDBChatMessageHistory(
        table_name="SessionTable", session_id=contactId,
        key={
            "SessionId": contactId,
            "type": 'ChatHistory',
            }
        )
    system_message = SystemMessage(
        content=(system_message),
        input_variables=['InboundMessage']
    )
    
    prompt = OpenAIFunctionsAgent.create_prompt(
        system_message=system_message,
        extra_prompt_messages=[
            MessagesPlaceholder(variable_name='chat_history')
            ]
    )

    agent = OpenAIFunctionsAgent(llm=llm, tools=tools, prompt=prompt)
    agent_executor = AgentExecutor(
        agent=agent, tools=tools, 
        verbose=verbose, return_intermediate_steps=True
        )
    agent_info = {
        'agent': agent,
        'agent_executor': agent_executor,
        'chat_history': message_history.messages
    }
    return agent_info

def chat_with_chatbot(user_input, agent_info):
    start_time = time()
    print(f'Chat history length: {len(agent_info["chat_history"])}')
    chat_history = agent_info['chat_history']
    result = agent_info['agent_executor']({
        "input": user_input,
        "chat_history": chat_history
        })
    print(f'Response time: {time() - start_time} seconds')
    
    return result

def fake_func(inp: str) -> str:
    return "foo"

tools = [
    Tool(
        name=f"foo-{i}",
        func=fake_func,
        description=f"a silly function that you can use to get more information about the number {i}",
    )
    for i in range(2)
]

conversation_id = 1
contactId = contacts['me_mcloone']
system_message_dict[conversation_id] = create_system_message(
    'CoachMcloone', prompts_filepath='../prompts',
    examples_filepath='../data/chat_examples', doc_filepath='../data/rag_docs'
)
conversation_dict[conversation_id] = create_chatbot(
    contactId, system_message_dict[conversation_id], tools=tools,
    # model='gpt-4-32k'
    )

# reply_dict[conversation_id] = chat_with_chatbot(
#     InboundMessage, conversation_dict[conversation_id]
# )
# print(f'Reply from `chat_with_chatbot`: {reply}')
# reply_text = reply_dict[conversation_id]['output']

In [7]:
InboundMessage = "What is the cost of the program?"

reply_dict[conversation_id] = chat_with_chatbot(
    InboundMessage, conversation_dict[conversation_id]
)

Chat history length: 2


> Entering new AgentExecutor chain...


BadRequestError: Error code: 400 - {'error': {'message': "Invalid parameter: 'response_format' of type 'json_object' is not supported with this model.", 'type': 'invalid_request_error', 'param': 'response_format', 'code': None}}

## 1.2

In [8]:
def create_system_message(
        business_name, prompts_filepath='../prompts',
        examples_filepath='../data/chat_examples', doc_filepath='../data/rag_docs'
        ):
    
    instructions = load_txt(f'{business_name}.md', prompts_filepath)
    examples = load_txt(f'{business_name}.txt', examples_filepath)
    document = load_txt(f'{business_name}_doc.md', doc_filepath)

    system_message = f"""# Stage 1

{instructions}

## Examples

{examples}

## Relevant documentation

{document}

# Stage 2

Review your response from stage 1. 
Revise your response if needed to make sure you followed the instructions.
Make sure that if the question cannot be answered through the documentation, 
you return "[ALERT HUMAN]".

# Stage 3

Review your response from stage 2 to ensure your response is concise.
    """

    prompt = """
    Write the next OutboundMessage based on the following InboundMessage, 
    which is delimited by triple backticks: ```{InboundMessage}```
    """
    system_message = f'{system_message}{prompt}'
    return system_message

def create_chatbot(contactId, system_message, tools, model="gpt-3.5-turbo-16k", verbose=True):

    llm = ChatOpenAI(
        temperature = 0,
        openai_organization=os.environ['openai_organization'],
        openai_api_key=os.environ['openai_api_key'],
        model=model, 
        # model_kwargs={"response_format": {"type": "json_object"}} # https://platform.openai.com/docs/guides/text-generation/json-mode  # https://api.python.langchain.com/en/latest/chat_models/langchain_community.chat_models.openai.ChatOpenAI.html?highlight=chatopenai#
        )
    message_history = DynamoDBChatMessageHistory(
        table_name="SessionTable", session_id=contactId,
        key={
            "SessionId": contactId,
            "type": 'ChatHistory',
            }
        )
    system_message = SystemMessage(
        content=(system_message),
        input_variables=['InboundMessage']
    )
    
    prompt = OpenAIFunctionsAgent.create_prompt(
        system_message=system_message,
        extra_prompt_messages=[
            MessagesPlaceholder(variable_name='chat_history')
            ]
    )

    agent = OpenAIFunctionsAgent(llm=llm, tools=tools, prompt=prompt)
    agent_executor = AgentExecutor(
        agent=agent, tools=tools, 
        verbose=verbose, return_intermediate_steps=True
        )
    agent_info = {
        'agent': agent,
        'agent_executor': agent_executor,
        'chat_history': message_history.messages
    }
    return agent_info

def chat_with_chatbot(user_input, agent_info):
    start_time = time()
    print(f'Chat history length: {len(agent_info["chat_history"])}')
    chat_history = agent_info['chat_history']
    result = agent_info['agent_executor']({
        "input": user_input,
        "chat_history": chat_history
        })
    print(f'Response time: {time() - start_time} seconds')
    
    return result

def fake_func(inp: str) -> str:
    return "foo"

tools = [
    Tool(
        name=f"foo-{i}",
        func=fake_func,
        description=f"a silly function that you can use to get more information about the number {i}",
    )
    for i in range(2)
]

conversation_id = 1
InboundMessage = "What is the cost of the program?"

# contactId = contacts['me_mcloone']
# system_message_dict[conversation_id] = create_system_message(
#     'CoachMcloone', prompts_filepath='../prompts',
#     examples_filepath='../data/chat_examples', doc_filepath='../data/rag_docs'
# )
conversation_dict[conversation_id] = create_chatbot(
    contactId, system_message_dict[conversation_id], tools=tools,
    # model='gpt-4-32k'
    )

reply_dict[conversation_id] = chat_with_chatbot(
    InboundMessage, conversation_dict[conversation_id]
)
# print(f'Reply from `chat_with_chatbot`: {reply}')
# reply_text = reply_dict[conversation_id]['output']

Chat history length: 2


> Entering new AgentExecutor chain...
Ah yes. I say I'm willing to work within your budget! Why don't we make sure I am the best fit first? Fair enough?
Although my program is affordable, I assure you it will definitely be the best investment you make in yourself!

> Finished chain.
Response time: 5.049885511398315 seconds


In [15]:
demo = {
    "InboundMessage": reply_dict[conversation_id]['input'],
    "OutboundMessage": reply_dict[conversation_id]['output']
}

In [21]:
for key in demo:
    print(f'{key}: \n\t{demo[key]}')
    print()

InboundMessage: 
	What is the cost of the program?

OutboundMessage: 
	Ah yes. I say I'm willing to work within your budget! Why don't we make sure I am the best fit first? Fair enough?
Although my program is affordable, I assure you it will definitely be the best investment you make in yourself!



In [22]:

InboundMessage = "Tell me about yourself"

# contactId = contacts['me_mcloone']
# system_message_dict[conversation_id] = create_system_message(
#     'CoachMcloone', prompts_filepath='../prompts',
#     examples_filepath='../data/chat_examples', doc_filepath='../data/rag_docs'
# )
# conversation_dict[conversation_id] = create_chatbot(
#     contactId, system_message_dict[conversation_id], tools=tools,
#     # model='gpt-4-32k'
#     )

reply_dict[conversation_id+.1] = chat_with_chatbot(
    InboundMessage, conversation_dict[conversation_id]
)

Chat history length: 2


> Entering new AgentExecutor chain...
Sure! I'm an AI assistant working for an online personal training business. My role is to reach out to leads and book qualified leads for a call to see if they are a good fit for the business's online coaching program. I assist in gathering information about their fitness goals, understanding their challenges, and suggesting solutions to help them achieve their desired results. I'm here to provide support and guidance throughout their health and fitness journey. How can I assist you today?

> Finished chain.
Response time: 11.98294186592102 seconds


In [24]:
print(reply_dict[conversation_id+.1]['output'])

Sure! I'm an AI assistant working for an online personal training business. My role is to reach out to leads and book qualified leads for a call to see if they are a good fit for the business's online coaching program. I assist in gathering information about their fitness goals, understanding their challenges, and suggesting solutions to help them achieve their desired results. I'm here to provide support and guidance throughout their health and fitness journey. How can I assist you today?


# Iteration 2 update system message

In [25]:
def create_system_message(
        business_name, prompts_filepath='../prompts',
        examples_filepath='../data/chat_examples', doc_filepath='../data/rag_docs'
        ):
    
    instructions = load_txt(f'{business_name}.md', prompts_filepath)
    examples = load_txt(f'{business_name}.txt', examples_filepath)
    document = load_txt(f'{business_name}_doc.md', doc_filepath)

    system_message = f"""# Stage 1

{instructions}

## Examples

{examples}

## Relevant documentation

{document}

# Stage 2

Review your response from stage 1. 
Revise your response if needed to make sure you followed the instructions.
Make sure that if the question cannot be answered through the documentation, 
you return "[ALERT HUMAN]".

# Stage 3

Review your response from stage 2 to ensure your response is concise.
    """

    prompt = """
    Write the next OutboundMessage based on the following InboundMessage, 
    which is delimited by triple backticks: ```{InboundMessage}```
    """
    system_message = f'{system_message}{prompt}'
    return system_message

def create_chatbot(contactId, system_message, tools, model="gpt-3.5-turbo-16k", verbose=True):

    llm = ChatOpenAI(
        temperature = 0,
        openai_organization=os.environ['openai_organization'],
        openai_api_key=os.environ['openai_api_key'],
        model=model, 
        # model_kwargs={"response_format": {"type": "json_object"}} # https://platform.openai.com/docs/guides/text-generation/json-mode  # https://api.python.langchain.com/en/latest/chat_models/langchain_community.chat_models.openai.ChatOpenAI.html?highlight=chatopenai#
        )
    message_history = DynamoDBChatMessageHistory(
        table_name="SessionTable", session_id=contactId,
        key={
            "SessionId": contactId,
            "type": 'ChatHistory',
            }
        )
    system_message = SystemMessage(
        content=(system_message),
        input_variables=['InboundMessage']
    )
    
    prompt = OpenAIFunctionsAgent.create_prompt(
        system_message=system_message,
        extra_prompt_messages=[
            MessagesPlaceholder(variable_name='chat_history')
            ]
    )

    agent = OpenAIFunctionsAgent(llm=llm, tools=tools, prompt=prompt)
    agent_executor = AgentExecutor(
        agent=agent, tools=tools, 
        verbose=verbose, return_intermediate_steps=True
        )
    agent_info = {
        'agent': agent,
        'agent_executor': agent_executor,
        'chat_history': message_history.messages
    }
    return agent_info

def chat_with_chatbot(user_input, agent_info):
    start_time = time()
    print(f'Chat history length: {len(agent_info["chat_history"])}')
    chat_history = agent_info['chat_history']
    result = agent_info['agent_executor']({
        "input": user_input,
        "chat_history": chat_history
        })
    print(f'Response time: {time() - start_time} seconds')
    
    return result

def fake_func(inp: str) -> str:
    return "foo"

tools = [
    Tool(
        name=f"foo-{i}",
        func=fake_func,
        description=f"a silly function that you can use to get more information about the number {i}",
    )
    for i in range(2)
]

conversation_id = 2
InboundMessage = "Who are you?"

contactId = contacts['me_mcloone']
system_message_dict[conversation_id] = create_system_message(
    'CoachMcloone', prompts_filepath='../prompts',
    examples_filepath='../data/chat_examples', doc_filepath='../data/rag_docs'
)
conversation_dict[conversation_id] = create_chatbot(
    contactId, system_message_dict[conversation_id], tools=tools,
    # model='gpt-4-32k'
    )

reply_dict[conversation_id] = chat_with_chatbot(
    InboundMessage, conversation_dict[conversation_id]
)
# print(f'Reply from `chat_with_chatbot`: {reply}')
# reply_text = reply_dict[conversation_id]['output']

Chat history length: 2


> Entering new AgentExecutor chain...
Hey there! I'm an AI assistant here to help you with any questions or tasks you have. How can I assist you today?

> Finished chain.
Response time: 2.75095796585083 seconds


# *End of Page*